<a href="https://colab.research.google.com/github/keyom-ai/prompts/blob/main/db_query_claude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is showcasing an example on how to generate SQL queries using Anthropic's Claude model.

## Simply ask natural language question and receive the output as SQL query.

Please add `.env` file in the root of the file system. Ensure that `.env` file has this line:

ANTHROPIC_API_KEY="sk-ant-api03-XXEwAA"

*replace with the right API key*

Install the SQLite database

In [1]:
%load_ext sql

In [2]:
%%sql sqlite:///test.db

In [3]:
%%sql create table sample(column_1 int, column_2 varchar);insert into sample values (1,'abc'),(2,'abcd');select * from sample;

 * sqlite:///test.db
Done.
2 rows affected.
Done.


column_1,column_2
1,abc
2,abcd


In [4]:
# Install the necessary libraries
%pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [5]:
# Import the required libraries
from anthropic import Anthropic
import sqlite3
import os

!pip install python-dotenv
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('.env')

# Set up the Anthropic API client
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"
api_key = os.environ["ANTHROPIC_API_KEY"]
client = Anthropic(api_key=api_key)

In [6]:
# Connect to the test database (or create it if it doesn't exist)
conn = sqlite3.connect("test_db.db")
cursor = conn.cursor()

# Create a sample table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        department TEXT,
        salary INTEGER
    )
""")

# Insert sample data
sample_data = [
    (1, "Jae Jones", "Sales", 50000),
    (2, "Raj Patel", "Engineering", 75000),
    (3, "Mike Johnson", "Sales", 60000),
    (4, "Emily Brown", "Engineering", 80000),
    (5, "Kai Lee", "Marketing", 55000)
]
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?)", sample_data)
conn.commit()

In [7]:
# Define a function to send a query to Claude and get the response
def ask_claude(query, schema):
    prompt = f"""Here is the schema for a database:

{schema}

Given this schema, can you output a SQL query to answer the following question? Only output the SQL query and nothing else.

Question: {query}
"""

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    )
    return response.content[0].text

In [8]:
# Get the database schema
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()
schema_str = "CREATE TABLE EMPLOYEES (\n" + "\n".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
print(schema_str)

CREATE TABLE EMPLOYEES (
id INTEGER
name TEXT
department TEXT
salary INTEGER
)


In [9]:
# Example natural language question
question = "What are the names and salaries of employees in the Engineering department?"
# Send the question to Claude and get the SQL query
sql_query = ask_claude(question, schema_str)
print(sql_query)


SELECT name, salary
FROM EMPLOYEES
WHERE department = 'Engineering';
